# Ex. 3.2 HBASE
We use the *HappyBase* lib: https://happybase.readthedocs.io/en/latest/installation.html

In [1]:
! conda install -c conda-forge -y happybase

Solving environment: ...working... done

## Package Plan ##

  environment location: F:\anaconda3

  added / updated specs:
    - happybase


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    happybase-1.2.0            |             py_0          26 KB  conda-forge
    thriftpy2-0.4.11           |   py37h8055547_0         239 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         265 KB

The following NEW packages will be INSTALLED:

  happybase          conda-forge/noarch::happybase-1.2.0-py_0
  thriftpy2          conda-forge/win-64::thriftpy2-0.4.11-py37h8055547_0

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1




thriftpy2-0.4.11     | 239 KB    |            |   0% 
thriftpy2-0.4.11     | 239 KB    | 6     

In [2]:
# imports
import happybase



In [18]:
# try to connect
server="ec2-3-250-8-167.eu-west-1.compute.amazonaws.com" #need to change this for every lab-session
connection = happybase.Connection(server)

In [13]:
print(connection.tables())

[b'AH_employees', b'JZ_employees', b'LG_employees', b'alex_id_employees', b'dnguyen_employees', b'dw_user_id_employees', b'ju_employees', b'lab3', b'mbaur_employees', b'mytable', b'steffen_id_employees', b'table_1_employees', b'table_s_employees', b'tizian_employees', b'user_id_employees', b'vkruck_employees']


In [12]:
#connect to a table
table = connection.table('lab3')

In [13]:
#see column families
table.families()

{b'family_1': {'name': b'family_1:',
  'max_versions': 3,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647},
 b'family_2': {'name': b'family_2:',
  'max_versions': 3,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [18]:
#insert something
table.put('row 3', {'family_2:col3': '17.33', 'family_2:col4': '5'})


TTransportException: TTransportException(type=4, message='TSocket read 0 bytes')

In [15]:
#read table
res=table.scan()

In [16]:
for key,row in res:
    print(key,row)

b'row 1' {b'family_1:col1': b'5', b'family_1:col2': b'string'}
b'row 2' {b'family_2:col1': b'5.564', b'family_2:col2': b'some text'}
b'row 3' {b'family_2:col3': b'17.33', b'family_2:col4': b'5'}
b'row 4' {b'family_1:col3': b'17.33', b'family_1:col4': b'5'}
b'row 6' {b'family_1:col7': b'17.33', b'family_1:col8': b'344'}
b'row 99' {b'family_2:col3': b'17.33', b'family_2:col4': b'5'}


# Ex 1
Employee data in HBASE.
* create a new table to hold the employee data -> use a distinct name, like ``user_id_employees``
* How would you create the column families?
* read data from ``employees.csv`` and put it into the table
* use batch-processing to enter more than one entry per transaction
* alter some entries of the table with ``timestamps``
* perform some sample scans for the latest entries

In [20]:
familie = {
    'employee' : dict()
}

connection.create_table('AH_employees', familie)

In [11]:
print(connection.tables())

TTransportException: TTransportException(type=4, message='TSocket read 0 bytes')

In [19]:
#connect to a table
table = connection.table('AH_employees')

In [5]:
table.families()

{b'employee': {'name': b'employee:',
  'max_versions': 3,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [51]:
import csv
with open('employees.csv', newline='') as csvfile:
    balitmore_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    
    batch = table.batch()
    
    count=0
    for row in balitmore_reader:
        values = row[0].split(',')
        batch.put('row' + str(count), {'employee:employee_id': values[0], 
                                  'employee:birthday': values[1], 
                                  'employee:first_name': values[2], 
                                  'employee:family_name': values[3], 
                                  'employee:gender': values[4], 
                                  'employee:workday': values[5]})
        count= count + 1
        
    batch.send()

In [24]:
#read table
res=table.scan('row0','row2')

In [25]:
for key,row in res:
    print(key,row)

TTransportException: TTransportException(type=4, message='TSocket read 0 bytes')